In [ ]:
!pip install emoji

In [ ]:
import re
import emoji

teencode_dict = {
    "hok": "không",
    "k": "không",
    "ko": "không",
    "dc": "được",
    "vl": "vãi",
    "thik": "thích",
    "bt": "bình thường",
    "j": "gì",
    "mik": "mình",
    "bn": "bạn",
    "vs": "với",
    "<3": "yêu"
}

def replace_teencode(text):
    text = text.lower()
    for teencode, replacement in teencode_dict.items():
        text = re.sub(r'\b' + re.escape(teencode) + r'\b', replacement, text)
    return text

def clean_text(text):
    text = emoji.replace_emoji(text, replace='')  # loại bỏ emoji
    text = replace_teencode(text)
    text = re.sub(r"…", "", text)
    text = re.sub(r"[^\w\s,.?!\-]", "", text)
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.lower()
    return text

# Test
text = "Ko bt mik thik cái này vl <3 😂 :))) www.google.com"
print(clean_text(text))


không bình thường mình thích cái này vãi 3


In [ ]:
!pip install pandas nltk random


ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [ ]:
import pandas as pd
import random
import json
import nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

# Đọc file txt
with open("/content/drive/MyDrive/NLP/data/word_net_vi.json", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Xử lý dữ liệu: tách các từ đồng nghĩa trên mỗi dòng
synonyms_list = []
for line in lines:
    # Bỏ khoảng trắng dư và tách theo dấu phẩy
    words = [word.strip() for word in line.strip().split(",") if word.strip()]
    if words:
        synonyms_list.append(words)

# Ghi ra file JSON
with open("/content/drive/MyDrive/NLP/data/word_net_vi.json", "w", encoding="utf-8") as f:
    json.dump(synonyms_list, f, ensure_ascii=False, indent=2)

print("Đã chuyển đổi thành công sang word_net_vi.json")


Đã chuyển đổi thành công sang word_net_vi.json


In [ ]:

# Nạp dữ liệu vào
data = pd.read_csv(r"/content/drive/MyDrive/NLP/data/processed_sentiment_data_no_emojis.csv")

# Cài đặt các phần phụ thuộc của NLTK
nltk.download('wordnet')
nltk.download('omw-1.4')

# Đọc file word_net__vi.json
with open(r'/content/drive/MyDrive/NLP/data/word_net_vi.json', 'r', encoding='utf-8') as f:
    word_net = json.load(f)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
def check_label_distribution(data):
    # Đếm số lượng các nhãn (sentiment)
    label_counts = data['sentiment'].value_counts()
    print("Phân bố nhãn trong dữ liệu:")
    print(label_counts)

    return label_counts

label_counts = check_label_distribution(data)

Phân bố nhãn trong dữ liệu:
sentiment
Tốt           1610
Tệ            1269
Trung tính     184
Name: count, dtype: int64


In [ ]:

# Hàm thêm từ đồng nghĩa vào câu (RI)
def random_insertion(text):
    words = text.split()

    # Chọn ngẫu nhiên một vị trí trong câu
    index = random.randint(0, len(words) - 1)

    # Lấy từ ở vị trí này
    word = words[index]

    # Kiểm tra xem từ này có trong word_net không
    if word in word_net and word_net[word]:  # Kiểm tra xem từ có đồng nghĩa không
        synonym = random.choice(word_net[word])
        words.insert(index, synonym)  # Thêm từ đồng nghĩa vào vị trí ngẫu nhiên

    return ' '.join(words)

# Hàm hoán đổi ngẫu nhiên hai từ trong câu (RS)
def random_swap(text):
    words = text.split()

    # Kiểm tra nếu có ít nhất 2 từ trong câu
    if len(words) < 2:
        return text  # Trả về câu gốc nếu có ít hơn 2 từ

    # Chọn ngẫu nhiên hai từ trong câu
    index1, index2 = random.sample(range(len(words)), 2)

    # Hoán đổi vị trí của hai từ
    words[index1], words[index2] = words[index2], words[index1]

    return ' '.join(words)

# Hàm tăng cường dữ liệu bằng cả hai kỹ thuật RI và RS
def augment_data(text, augmentation_type='both'):
    if augmentation_type == 'RI':
        return random_insertion(text)
    elif augmentation_type == 'RS':
        return random_swap(text)
    elif augmentation_type == 'both':
        return random_swap(random_insertion(text))
    else:
        return text

# Cân bằng dữ liệu cho nhãn "Trung tính"
def balance_neutral_data(data, target_count):
    # Lọc các câu có nhãn 'Trung tính'
    neutral_data = data[data['sentiment'] == 'Trung tính']

    # Số lượng cần thêm cho nhãn 'Trung tính'
    diff = target_count - len(neutral_data)

    augmented_data = neutral_data.copy()

    for _ in range(diff):
        # Chọn ngẫu nhiên một câu và tăng cường nó
        row = neutral_data.sample(1)
        augmented_text = augment_data(row['comment'].values[0], augmentation_type='both')
        augmented_data = pd.concat([augmented_data,
                                    pd.DataFrame({ 'comment': [augmented_text],
                                                  'sentiment': ['Trung tính']})], ignore_index=True)

    return augmented_data




In [ ]:
# Số lượng mẫu mục tiêu cho nhãn 'Trung tính'
target_neutral_count = 1000  # Ví dụ bạn muốn có 1000 câu cho nhãn 'Trung tính'

# Cân bằng dữ liệu cho nhãn "Trung tính"
balanced_neutral_data = balance_neutral_data(data, target_neutral_count)

# Thêm lại dữ liệu đã cân bằng vào dữ liệu gốc
balanced_data = pd.concat([data[data['sentiment'] != 'Trung tính'], balanced_neutral_data], ignore_index=True)

# Hiển thị dữ liệu sau khi cân bằng
print("Dữ liệu sau khi cân bằng nhãn Trung tính:")
print(balanced_data['sentiment'].value_counts())

# Lưu dữ liệu đã cân bằng
balanced_data.to_csv("/content/drive/MyDrive/NLP/data/balanced_neutral_sentiment_data.csv", index=False)

Dữ liệu sau khi cân bằng nhãn Trung tính:
sentiment
Tốt           1610
Tệ            1269
Trung tính    1000
Name: count, dtype: int64
